<a href="https://colab.research.google.com/github/makezazainw/AESexample/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Upload file if you want**

In [ ]:
from google.colab import files
pbfile = files.upload()

**Install Kaggle**

In [1]:
# Install Kaggle API
!pip install -q kaggle
!pip install -q kaggle-cli

     |████████████████████████████████| 81kB 2.5MB/s 
     |████████████████████████████████| 5.3MB 4.5MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 112kB 40.5MB/s 
     |████████████████████████████████| 143kB 42.5MB/s 


**Call file from Json for use Kaggle Dataset**



In [2]:
# only for google colab
import os
os.environ['KAGGLE_USERNAME'] = "thammatattantipitham" 
os.environ['KAGGLE_KEY'] = "3cf826ab97707228fb1602519992740b"

In [3]:
!kaggle datasets download -d thammatattantipitham/originalthaiherb --unzip

 87% 13.0M/15.0M [00:01<00:00, 5.54MB/s]
100% 15.0M/15.0M [00:01<00:00, 10.7MB/s]


**Move to Directory thai-herb**

In [4]:
!mkdir originalthaiherb
!mv  "train" "val" originalthaiherb

**Start**

In [5]:
import sys
import time
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
import keras

from skimage.transform import resize
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.optimizers import Adam
from keras.applications import InceptionV3
sys.version

'3.6.9 (default, Jul 17 2020, 12:50:27) \n[GCC 8.4.0]'

In [6]:
from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7



In [8]:
DATASET_PATH  = 'originalthaiherb/'
IMAGE_SIZE    = (32, 32)
NUM_CLASSES   = 10
BATCH_SIZE    = 32  # try reducing batch size or freeze more layers if your GPU runs out of memory
epochs    = 200
WEIGHTS_FINAL = 'save_model/model-resnet50-final.h5'

In [9]:
#ทำ aug มาเเล้ว
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [10]:
#set training data
train_generator = train_datagen.flow_from_directory(DATASET_PATH+ r"train/",
                                                    target_size = IMAGE_SIZE,
                                                    batch_size = BATCH_SIZE,
                                                    class_mode='categorical',
                                                    shuffle = True,
                                                    subset='training')

Found 1600 images belonging to 10 classes.


In [11]:
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_generator = valid_datagen.flow_from_directory(DATASET_PATH+ r"val/",                                      
                                                    target_size = IMAGE_SIZE,
                                                    batch_size = 1,
                                                    class_mode='categorical',
                                                    shuffle = True
                                                    )

Found 410 images belonging to 10 classes.


In [12]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=100)

#Create the model
base_model_resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3),classes=valid_generator)
#Defining and Adding layers
model_resnet= Sequential()
#Add the Dense layers along with activation and batch normalization
model_resnet.add(base_model_resnet)
model_resnet.add(Flatten())

#Add the Dense layers along with activation and batch normalization
model_resnet.add(Dense(1024,activation=('relu'),input_dim=64))
model_resnet.add(Dense(512,activation=('relu'))) 
model_resnet.add(Dropout(.4))
model_resnet.add(Dense(256,activation=('relu'))) 
model_resnet.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights
model_resnet.add(Dense(128,activation=('relu')))
model_resnet.add(Dropout(.2))
model_resnet.add(Dense(10,activation=('softmax'))) #This is the classification layer


# Save the weights using the `checkpoint_path` format
model_resnet.save_weights(checkpoint_path.format(epoch=0))

94773248/94765736 [==============================] - 1s 0us/step


In [13]:
#Model summary
model_resnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0

In [14]:
model_resnet.compile(optimizer=Adam(lr=0.0001),  #Adam(lr=1e-5)
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
#start to train the model
start = time.time()
model_resnet.fit_generator(generator=train_generator, epochs=epochs, steps_per_epoch=train_generator.samples//BATCH_SIZE, validation_data=valid_generator,validation_steps=valid_generator.samples,callbacks=[cp_callback],verbose=1)

end = time.time()
duration = end - start
print ('\n model_ResNet50 took %0.2f seconds (%0.1f minutes) to train for %d epochs'%(duration, duration/60, epochs) )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
50/50 [==============================] - 13s 267ms/step - loss: 2.4085 - accuracy: 0.1587 - val_loss: 2.1775 - val_accuracy: 0.2439
Epoch 2/200
50/50 [==============================] - 12s 244ms/step - loss: 1.8551 - accuracy: 0.3481 - val_loss: 1.8961 - val_accuracy: 0.4073
Epoch 3/200
50/50 [==============================] - 12s 237ms/step - loss: 1.4374 - accuracy: 0.5163 - val_loss: 1.4441 - val_accuracy: 0.5976
Epoch 4/200
50/50 [==============================] - 12s 240ms/step - loss: 1.0679 - accuracy: 0.6637 - val_loss: 1.0328 - val_accuracy: 0.7463
Epoch 5/200
50/50 [==============================] - 12s 241ms/step - loss: 0.8167 - accuracy: 0.7337 - val_loss: 0.6665 - val_accuracy: 0.8244
Epoch 6/200
50/50 [==============================] - 12s 241ms/step - loss: 0.6178 - accuracy: 0.7994 - val_loss: 0.4754 - val_accuracy: 0.8707
Epoch 7/200
50/50 [==============================] - 12s 239

In [16]:
!mkdir save_model
model_resnet.save_weights('save_model/model-resnet50(200epochs).h5')

In [48]:
model_resnet.history.history.keys()

dict_keys([])

In [49]:
#กราฟเเม่นยำเเละ loss
import matplotlib.pyplot as plt

acc = model_resnet.history.history['accuracy']
val_acc = model_resnet.history.history['val_accuracy']

loss= model_resnet.history.history['loss']
val_loss= model_resnet.history.history['val_loss']


plt.style.use("ggplot")
#  "Accuracy"
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
#  "Loss"
plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.show()
# plt.savefig('accandloss.png')

KeyError: ignored

In [67]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0200.ckpt'

In [71]:
# Save the entire model as a SavedModel.
model_resnet.save('saved_model/my_model') 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [72]:
!ls saved_model/mymodel

my_model


In [68]:
# Load the previously saved weights
model_resnet.load_weights(latest)

# Re-evaluate the model
loss, acc = model_resnet.evaluate(valid_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

410/410 - 7s - loss: 0.2364 - accuracy: 0.9488
Restored model, accuracy: 94.88%


In [73]:
model= tf.keras.models.load_model('saved_model/my_model')
tflite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tflite_converter.convert()
open("tf_lite_model.tflite", "wb").write(tflite_model) 

INFO:tensorflow:Assets written to: /tmp/tmp54fzncnj/assets


105133220

**Prediction**

In [59]:
model_resnet.load_weights('save_model/model-resnet50(200epochs).h5')